In [55]:
# Specify variables
# sys.path.insert(local_haven)
from haven import haven_jupyter as hj
from haven import haven_results as hr
from haven import haven_utils as hu

# please define the path to the experiments
savedir_base = '/mnt/projects/vision_prototypes/synbols/logs_borgy'
savedir_base = '/mnt/datasets/public/research/pau/pau-ousmane/logs_celeba_borgy/'

exp_list = None

# exp_config_name = <exp_config_name>
# exp_list = hu.load_py(exp_config_name).EXP_GROUPS['mnist']

# get specific experiments, for example, {'model':'resnet34'}
filterby_list = None

# group the experiments based on a hyperparameter, for example, ['dataset']
groupby_list = None
verbose = 0

# plot vars
y_metrics='train_loss'
x_metric='epoch'
log_metric_list = ['train_loss']
map_exp_list = []
title_list=['dataset']
legend_list=['model']

# get experiments
rm = hr.ResultManager(exp_list=exp_list, 
                      savedir_base=savedir_base, 
                      filterby_list=filterby_list,
                      verbose=verbose
                     )
score_lists = rm.get_score_lists()
# score = rm.get_score_table(columns=['dataset', 'exp_id'])


# launch dashboard
hj.get_dashboard(rm, vars(), wide_display=True)

100%|██████████| 108/108 [00:00<00:00, 847.86it/s]


<IPython.core.display.Javascript object>

Output()

In [51]:
import numpy as np

exp_list = rm.get_exp_list_df()

def get_hyperparameters(exp_list_df, min_threshold=2, max_threshold=None):
    exp_list_df.reset_index()
    if max_threshold is None:
        max_threshold = exp_list.shape[0] - 1
    column_count = []
    for column in exp_list.columns:
        _set = set([str(v) for v in exp_list[column].values])
        column_count.append(len(_set))
    indices = np.arange(len(exp_list.columns))
    column_count = np.array(column_count)
    indices = indices[(column_count >= min_threshold) & (column_count <= max_threshold)]
    hyperparameters = [exp_list.columns[i] for i in indices]
    return hyperparameters

In [52]:
print(get_hyperparameters(exp_list, max_threshold=5))

[]


In [ ]:
# aggregate results
result_list = []
for exp_dict in rm.exp_list:
    total_nested = 0
    for k, v in exp_dict.items():
        if isinstance(v, dict):
            total_nested += 1
    while total_nested > 0:
        keys = exp_dict.keys()
        for k in keys:
            v = exp_dict[k]
            if insinstance(v, dict):
                for k2, v2 in v.items():
                    exp_dict["%s_%s" %(k, k2)] = v2
                    if isinstance(v2, dict):
                        total_nested += 1
                total_nested -= 1
                del(exp_dict[k])
                
    result_dict = {}

    exp_id = hu.hash_dict(exp_dict)
    result_dict["exp_id"] = exp_id

    for k in exp_dict:
        result_dict[k] = exp_dict[k]

    result_list += [result_dict]
        
pd.DataFrame(result_list)